# Initialize Variables

In [2]:
from repoextractor import RepoExtractor
from repovalidator import RepoValidator
import json
import os

ORG = "wikimedia"

# Get the list of repositories (`processed/ORG/repos.json`)

In [ ]:

RepoExtractor().extract_repo_names(ORG)


# Extract raw commits (`raw/ORG/REPO.commits.json`)


In [2]:
import datetime

today = datetime.datetime.today()
deadline = datetime.timedelta(days=365*2)
since = today - deadline

with open(f"../data/processed/{ORG}/repos.json", "r") as file:
    data = json.load(file)
    
    start_index = 0
    for i, repo in enumerate(data):
        if repo["name"] == "mediawiki-extensions-Collection":
            start_index = i
            break

    for repo in data[start_index + 1:]:
        print(repo["name"])
        RepoExtractor().extract_raw_commits_from_repo(repo["name"], since)
    
    

wikimedia/wikipedia-ios



# Extract commit messages and dates (`processed/ORG/commits-messages-dates/REPO.pickle`)


In [3]:
for subdir, dirs, files in os.walk(f"../data/raw/{ORG}"):
    for file in files:
        repo = file.split(".")[0]
        path = os.path.join(subdir, file)
        RepoExtractor().save_commits_messages_dates(path,
                                                    ORG,
                                                    repo)
        



# Extract repos with at least 2 commits per month (`processed/ORG/months-valid.json`)


In [4]:
repos_bool = []
for subdir, dirs, files in os.walk(f"../data/processed/{ORG}/commits-messages-dates"):
    for file in files:
        filename = file.split(".")[0]
        complete_path = os.path.join(subdir, file)
        repos_bool.append({"name": filename, "is_valid": RepoValidator(complete_path).has_at_least_2_commits_per_month()})
        
with open(f"../data/processed/{ORG}/months-valid.json", "w") as f:
    valids = list(map(lambda y: y["name"], filter(lambda x: x["is_valid"], repos_bool)))
    json.dump(valids, f)

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
                                                                                    message  \
e71cee92f3c8273749b747a155f802e62a425d0a                              Add link to wiki page   
b0da55ab9358c2c59822e6bf5584038745a69434  Merge pull request #31 from wikimedia/develop\...   
9cb92ee8bedbbcbfb93ef0fedd5496fdedfea684                                           Fix typo   
5adece1ab6a96a7d91800565327ba480c42bc2c8  Updated README with further improvements and i...   
bb6d51d796045d9340bcaca1653f350adc29a724  Merge pull request #30 from wikimedia/web-serv...   

                                                          date  
e71cee92f3c8273749b747a155f802e62a425d0a  2021-03-11T19:21:38Z  
b0da55ab9358c2c59822e6bf5584038745a69434  2021-03-11T19:11:59Z  
9cb92ee8bedbbcbfb93ef0fedd5496fdedfea684  2021-03-11T19:11:34Z  
5adece1ab6a96a7d91800565327ba480c42bc2c8  2021-03-10T21:29:48Z  
bb6d51d796045d9340bcaca1653f

c:\Users\m_bla\Desktop\Code_Poly\LOG6307\log6307e-replication-project\src\repovalidator.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df.loc[:, 'month'] = pd.to_datetime(self.df.date).dt.month
c:\Users\m_bla\Desktop\Code_Poly\LOG6307\log6307e-replication-project\src\repovalidator.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df.loc[:, 'year'] = pd.to_datetime(self.df.date).dt.year


                                                                                    message  \
f369196d116f1c86c455b47db92046d7373df83e  Merge "Improve reference handling in html adap...   
a5b08bf1726b79a0c25c25c9d6bc05107b49f2f1  Improve reference handling in html adaptation\...   
fc004cf560246470e67485c5bfb5264f90e7eff8  Avoid exception when invalid language code is ...   
b5ea2d74801e6ee658a1c72880e123df0e03ab40  ui: Display error message when translation fai...   
99da8cd3db94c7c0acbe2a6acaa7bc7c8f13108e  Use language identification service, remove py...   

                                                          date  
f369196d116f1c86c455b47db92046d7373df83e  2023-10-25T03:29:20Z  
a5b08bf1726b79a0c25c25c9d6bc05107b49f2f1  2023-10-19T07:06:21Z  
fc004cf560246470e67485c5bfb5264f90e7eff8  2023-10-18T05:36:47Z  
b5ea2d74801e6ee658a1c72880e123df0e03ab40  2023-10-18T05:06:28Z  
99da8cd3db94c7c0acbe2a6acaa7bc7c8f13108e  2023-10-18T04:18:32Z  
Empty DataFrame
Columns: []
Index: []
 


# Extract commits with their files (`/processed/ORG/commits-files/REPO.commits.pickle`)


In [1]:
from repoextractor import RepoExtractor
from repovalidator import RepoValidator
import json
import os
import datetime

ORG = "wikimedia"
today = datetime.datetime.today()
delta = datetime.timedelta(days=365*2)
since = today - delta

with open(f"../data/processed/{ORG}/months-valid.json", "r") as file:
    extractor = RepoExtractor()
    data = json.load(file)
    start_index = 0
    for i, repo in enumerate(data):
        if repo == "apps-android-wikipedia":
            start_index = i
            break

    for repo in data[start_index:]: 
        r = extractor.get_repo(f"{ORG}/{repo}")
        extractor.extract_repo(r, ORG, since=since)
        
        


TypeError: function missing required argument 'year' (pos 1)


# Extract repos with at least 11% of IaC files (`processed/ORG/valid-repos.json`)


In [1]:
valids = []
for subdir, dirs, files in os.walk(f"../data/processed/{ORG}/commits-files"):
    for file in files:
        repo = file.split(".")[0]
        path = os.path.join(subdir, file)
        if RepoValidator(path).has_11_percent_of_iac():
            valids.append(repo)
valids            

                                                                        files
012d4a172b42df76ecde09c70d1608f5f687099c                 [artifactory.groovy]
01b42818532126c8e34742a874cc992892e91129                      [debian.groovy]
046fb06d88d573d14a136bdbd1d72b643b0accb1                      [debian.groovy]
0b8933a4662b016de61a2f6e947ab6e9fb4d39cc                        [salt.groovy]
0d3d8ae391d29283f041b2d8bce758f996e3adcf  [artifactory.groovy, docker.groovy]
                                                                                      files
005ceaa8b481d7082e078f4d7219195f53d44eb0  [roles/testnode/tasks/yum/packages.yml, roles/...
00e8e4bd2477c13322329eee62c5461f65ec1268  [roles/gateway/defaults/main.yml, roles/gatewa...
027e1ec7583f43f692aeb4c439fab2b1096d2679  [roles/cobbler/templates/triggers/install/post...
02f9248fc6ed1861477076fdfe7c90e1fc503aae  [roles/testnode/vars/apt_systems.yml, roles/te...
0373f93b02aa12f533d0323c741a12f1799d45ea             [roles/gateway/temp

[]

In [ ]:
with open(f'../data/processed/{ORG}/valid-repos.json', "w") as f:
    json.dump(valids, f)